In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, pipeline  
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from datasets import Dataset 
import pandas as pd
import numpy as np
import torch
import evaluate 
import os 
import torch
import gradio as gr



In [2]:
ft_model = os.path.abspath('./models/optimized_model')


In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [4]:
import os
from dotenv import load_dotenv

load_dotenv()
token = os.getenv("HUGGINGFACE_TOKEN")


In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
import os
from dotenv import load_dotenv
from huggingface_hub import HfApi

# Load environment variables
load_dotenv()

# Securely fetch the token
token = os.getenv("HUGGINGFACE_TOKEN")

# Initialize API with token
api = HfApi(token=token)

# Upload your model folder
api.upload_folder(
    folder_path="/Users/erum/LHL_LLM/notebooks/optimized_model",
    repo_id="efarooqi/sentiment_analysis",
    repo_type="model",
)




Uploading...:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/efarooqi/sentiment_analysis/commit/887ba5b983cd018517e97fa14c1b823fd914ea09', commit_message='Upload folder using huggingface_hub', commit_description='', oid='887ba5b983cd018517e97fa14c1b823fd914ea09', pr_url=None, repo_url=RepoUrl('https://huggingface.co/efarooqi/sentiment_analysis', endpoint='https://huggingface.co', repo_type='model', repo_id='efarooqi/sentiment_analysis'), pr_revision=None, pr_num=None)

In [7]:


# Load model and tokenizer from your saved directory
model_path = "/Users/erum/LHL_LLM/notebooks/optimized_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Text to test
test_text = "This is absolutely fantastic movie"

# Tokenize the input text
inputs = tokenizer(test_text, return_tensors="pt", padding=True, truncation=True)

# Run prediction
model.eval()
with torch.no_grad():
    logits = model(**inputs).logits
    predicted_class = torch.argmax(logits, dim=1).item()

# Interpret the predicted class
label_map = model.config.id2label
print("Predicted Sentiment:", label_map[predicted_class])


Predicted Sentiment: Positive


In [8]:


# Load model and tokenizer
model_path = "/Users/erum/LHL_LLM/notebooks/optimized_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()

# Prediction function
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_class = torch.argmax(logits, dim=1).item()
        label_map = model.config.id2label
        return label_map[predicted_class]



In [10]:
# Launch Gradio UI
demo = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(lines=2, placeholder="Enter movie review..."),
    outputs="label",
    title="Sentiment Analysis",
    description="Enter a movie review to see whether it's classified as Positive or Negative 🎬"
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
